In [5]:
import pandas as pd

# Read both sheets
df_photos = pd.read_excel('photo_repeat_group.xlsx', sheet_name='photo_repeat_group')
df_tree_numbers = pd.read_excel('photo_repeat_group.xlsx', sheet_name='tree number to urls match')

# Look at the first few rows of both dataframes to understand the structure
print("Tree numbers sheet first few rows:")
print(df_tree_numbers[['submission_id', 'tree_number']].head(10))

print("\
Photo repeat group sheet first few rows:")
print(df_photos[['parent_index', 'tree_url']].head(10))

Tree numbers sheet first few rows:
   submission_id  tree_number
0      405296368            1
1      405296368            5
2      405296368            9
3      405296584            3
4      405296584            6
5      405296687            3
6      405296687            6
7      405296687           10
8      405296844            1
9      405296844            7
Photo repeat group sheet first few rows:
   parent_index                                           tree_url
0             1  https://kc.kobotoolbox.org/media/original?medi...
1             1  https://kc.kobotoolbox.org/media/original?medi...
2             1  https://kc.kobotoolbox.org/media/original?medi...
3             1  https://kc.kobotoolbox.org/media/original?medi...
4             1  https://kc.kobotoolbox.org/media/original?medi...
5             1  https://kc.kobotoolbox.org/media/original?medi...
6             1  https://kc.kobotoolbox.org/media/original?medi...
7             1  https://kc.kobotoolbox.org/media/original

In [7]:
# Get unique parent indices for each submission_id and tree_number combination
unique_indices = df_photos['parent_index'].unique()
print("\
Number of unique parent indices:", len(unique_indices))

# Create a mapping dictionary based on the sequence of appearance in the tree_numbers sheet
mapping = {}
current_parent_index_idx = 0

for _, row in df_tree_numbers.iterrows():
    submission_tree_key = (row['submission_id'], row['tree_number'])
    if submission_tree_key not in mapping and current_parent_index_idx < len(unique_indices):
        mapping[submission_tree_key] = unique_indices[current_parent_index_idx]
        current_parent_index_idx += 1

# Function to get URLs for a specific submission_id and tree_number
def get_urls_for_tree(submission_id, tree_number):
    parent_idx = mapping.get((submission_id, tree_number))
    if parent_idx is not None:
        urls = df_photos[df_photos['parent_index'] == parent_idx]['tree_url'].tolist()
        return ','.join(urls) if urls else None
    return None

# Create the tree_urls column
df_tree_numbers['tree_urls'] = df_tree_numbers.apply(
    lambda row: get_urls_for_tree(row['submission_id'], row['tree_number']), 
    axis=1
)

# Save the updated dataframe
output_filename = 'tree_numbers_with_sequential_mapping.xlsx'
df_tree_numbers.to_excel(output_filename, index=False)

# Print the first few mappings to verify
print("\
First 10 mappings:")
for i, ((submission_id, tree_number), parent_idx) in enumerate(list(mapping.items())[:10]):
    print(f"Submission {submission_id}, Tree {tree_number} -> Parent index {parent_idx}")

print("\
Sample of updated dataframe:")
print(df_tree_numbers.head())

Number of unique parent indices: 141
First 10 mappings:
Submission 405296368.0, Tree 1.0 -> Parent index 1
Submission 405296368.0, Tree 5.0 -> Parent index 2
Submission 405296368.0, Tree 9.0 -> Parent index 3
Submission 405296584.0, Tree 3.0 -> Parent index 11
Submission 405296584.0, Tree 6.0 -> Parent index 12
Submission 405296687.0, Tree 3.0 -> Parent index 21
Submission 405296687.0, Tree 6.0 -> Parent index 22
Submission 405296687.0, Tree 10.0 -> Parent index 23
Submission 405296844.0, Tree 1.0 -> Parent index 31
Submission 405296844.0, Tree 7.0 -> Parent index 32
Sample of updated dataframe:
   submission_id  tree_number  \
0      405296368            1   
1      405296368            5   
2      405296368            9   
3      405296584            3   
4      405296584            6   

                                           tree_urls  
0  https://kc.kobotoolbox.org/media/original?medi...  
1  https://kc.kobotoolbox.org/media/original?medi...  
2  https://kc.kobotoolbox.org/med